# Assignment 2: Retrieval-Augmented Question Answering with LLMs

In this assignment, we will use a large language model (LLM) in a retrieval-augmented setup to answer questions from the Natural Questions dataset. We will evaluate the performance of the LLM using different prompting strategies and compare the results. The steps involved are as follows:

1. **Evaluate an LLM on Natural Questions without context.**
2. **Evaluate an LLM on Natural Questions with provided context.**
3. **Set up a retriever to find relevant passages.**
4. **Evaluate the LLM using retrieved context instead of provided context.**


In [1]:
import pandas as pd
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

nq_data = pd.read_csv('nq_simplified.val.tsv', sep='\t', header=None, names=['question', 'answer', 'gold_context'], quoting=3)

nq_data.head()

,question,answer,gold_context
0,what purpose did seasonal monsoon winds have o...,enabled European empire expansion into the Ame...,The westerlies (blue arrows) and trade winds (...
1,who got the first nobel prize in physics,"Wilhelm Conrad Röntgen, of Germany",The award is presented in Stockholm at an annu...
2,when is the next deadpool movie being released,"May 18, 2018","Though the original creative team of Reynolds,..."
3,where did the idea of fortnite come from,as a cross between Minecraft and Left 4 Dead,"Fortnite is set in contemporary Earth, where t..."
4,which mode is used for short wave broadcast se...,MFSK Olivia,"All one needs is a pair of transceivers, each ..."


In [2]:
!nvidia-smi

Fri May 17 13:24:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4080        Off | 00000000:21:00.0  On |                  N/A |
|  0%   36C    P8              11W / 320W |   1259MiB / 16376MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
def rouge1(gold, predicted):
    assert(len(gold) == len(predicted))
    n_p = 0
    n_g = 0
    n_c = 0
    for g, p in zip(gold, predicted):
        g = set(cleanup(g).strip().split())
        p = set(cleanup(p).strip().split())
        n_g += len(g)
        n_p += len(p)
        n_c += len(p.intersection(g))
    pr = n_c / n_p
    re = n_c / n_g
    if pr > 0 and re > 0:
        f1 = 2*pr*re/(pr + re)
    else:
        f1 = 0.0
    return pr, re, f1

def cleanup(text):
    text = text.replace(',', ' ')
    text = text.replace('.', ' ')
    return text

## Step 1: Evaluating an LLM on Natural Questions

First, we will find an LLM on huginface hub that is small and fast to fitt on our system. 

In [4]:
"""
Load the model and tokenizer
 "mistralai/Mixtral-8x7B-Instruct-v0.1" "microsoft/Phi-3-mini-4k-instruct" "mtgv/MobileLLaMA-1.4B-Base" "TheBloke/Mistral-7B-Instruct-v0.2-AWQ" 
"TheBloke/Llama-2-7B-Chat-AWQ"

"""
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-AWQ" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
    low_cpu_mem_usage=True,
    device_map="cuda:0")


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.15G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Lets create a simple pipline for QA and use a small subset of the dataset to get the predicted answers and evaluate the model using ROUGE-1 scores.

In [5]:
%%time
# Create a pipeline for question answering
qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, return_full_text=False, 
                       pad_token_id=tokenizer.eos_token_id, max_new_tokens=400,)

# Define a function to get answers from the model
def get_answers(questions, max_new_tokens=400):
    answers = []
    for question in questions:
        response = qa_pipeline(question)
        answers.append(response[0]['generated_text'])
    return answers

# Use a small subset of the dataset for testing
subset = nq_data.sample(20)
questions = subset['question'].tolist()
gold_answers = subset['answer'].tolist()

predicted_answers = get_answers(questions)

# Evaluate the model
pr, re, f1 = rouge1(gold_answers, predicted_answers)
print(f"ROUGE-1 Precision: {pr}, Recall: {re}, F1 Score: {f1}")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


ROUGE-1 Precision: 0.013316423589093214, Recall: 0.3333333333333333, F1 Score: 0.025609756097560974
CPU times: user 2min 51s, sys: 20 ms, total: 2min 51s
Wall time: 2min 50s


## Step 2: An Idealized Retrieval-Augmented LLM

Now, we will include the `gold_context` in our prompts and evaluate the model again. This will help us understand the upper limits of the system.


In [6]:
%%time
# Define a function to get answers from the model with context
def get_answers_with_context(questions, contexts):
    answers = []
    for question, context in zip(questions, contexts):
        prompt = f"Question: {question}\nContext: {context}\nAnswer:"
        response = qa_pipeline(prompt)
        answers.append(response[0]['generated_text'])
    return answers

contexts = subset['gold_context'].tolist()

predicted_answers_with_context = get_answers_with_context(questions, contexts)

pr_context, re_context, f1_context = rouge1(gold_answers, predicted_answers_with_context)
print(f"With Context - ROUGE-1 Precision: {pr_context}, Recall: {re_context}, F1 Score: {f1_context}")


With Context - ROUGE-1 Precision: 0.08614864864864864, Recall: 0.8095238095238095, F1 Score: 0.15572519083969466
CPU times: user 36 s, sys: 0 ns, total: 36 s
Wall time: 36 s


## Step 3: Setting Up the Retriever

In this step, we will set up a retriever to find relevant passages. We will use the SentenceTransformers model to create embeddings for the passages and set up a FAISS index for efficient retrieval.


In [7]:
#!pip install faiss-cpu sentence_transformers

In [8]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

retriever_model = SentenceTransformer('all-MiniLM-L6-v2')

with open('passages.txt', 'r', encoding='utf-8') as file:
    passages = file.readlines()

embedded_passages = retriever_model.encode(passages, convert_to_tensor=True)

index = faiss.IndexFlatL2(embedded_passages.shape[1])
index.add(embedded_passages.cpu())


In [9]:
def retrieve_best_passage(question):
    question_embedding = retriever_model.encode([question], convert_to_tensor=True)
    _, ix = index.search(question_embedding.cpu(), 1)
    return passages[ix[0][0]]

question = "Where did the first African American air force unit train?"
best_passage = retrieve_best_passage(question)
print(f"Best Passage: {best_passage}")


Best Passage: Black Americans were not permitted to fly for the U.S. armed services prior to 1940. The Air Corps at that time, which had never had a single black member, was part of an army that possessed exactly two black Regular line officers at the beginning of World War II: Brigadier Generals Benjamin O. Davis, Sr. and Benjamin O. Davis, Jr. The first Civilian Pilot Training Program (CPTP) students completed their instruction in May 1940. The creation of an all-black pursuit squadron resulted from pressure by civil rights organizations and the black press who pushed for the establishment of a unit at Tuskegee, an



## Step 4: Putting the Pieces Together

Finally, we will retrieve passages for each question and evaluate the model using these passages instead of the provided `gold_context`.

In [10]:
def get_answers_with_retrieved_context(questions):
    answers = []
    for question in questions:
        context = retrieve_best_passage(question)
        prompt = f"Question: {question}\nContext: {context}\nAnswer:"
        response = qa_pipeline(prompt)
        answers.append(response[0]['generated_text'])
    return answers

predicted_answers_retrieved_context = get_answers_with_retrieved_context(questions)

pr_retrieved, re_retrieved, f1_retrieved = rouge1(gold_answers, predicted_answers_retrieved_context)
print(f"With Retrieved Context - ROUGE-1 Precision: {pr_retrieved}, Recall: {re_retrieved}, F1 Score: {f1_retrieved}")


With Retrieved Context - ROUGE-1 Precision: 0.030193236714975844, Recall: 0.3968253968253968, F1 Score: 0.056116722783389444


## Summary

In this notebook, we evaluated the performance of an LLM on the Natural Questions dataset using different prompting strategies. We observed the following:

1. **Without Context**:
   - ROUGE-1 Precision: *result*
   - Recall: *result*
   - F1 Score: *result*

2. **With Provided Context**:
   - ROUGE-1 Precision: *result*
   - Recall: *result*
   - F1 Score: *result*

3. **With Retrieved Context**:
   - ROUGE-1 Precision: *result*
   - Recall: *result*
   - F1 Score: *result*

Including context, whether provided or retrieved, ...
